# Tutorial: Xilinx Virtual Cable (XVC) with PYNQ

In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import pynq
import time

os.chdir('/home/xilinx/jupyter_notebooks/xvc_demo/PYNQ_XilinxVirtualCable/jtag/zynq7000_PYNQ')

pynq.PL.reset()

# The following will load the FW bit file
# Note: the (*.hwh) must be in the same directory with
# the exact same name, except for the file extension...
ol = pynq.Overlay('XVC_Demo.bit')

## Demo FW Block Diagram

![Block Diagram](Block_Diagram.png) 

In [ ]:
# This will show the ip_dict in a nice searchable view...
ol.ip_dict

In [ ]:
import pynq_xvc_utils as pxvc

# The following function will scan the PYNQ Overlay and launch XVC Server
# instances (PYNQ Memory Mapped) for any Debug Bridge IP if finds that use
# either of the following modes:
#     
#     AXI-to-BSCAN   (For debuging design in the same PL)
#         or
#     AXI-to-JTAG    (For debugging a seperate external FPGA Part using physical JTAG pins, multiple additional external JTAG chanis can be addeded !!!)

xvc_list = pxvc.start_servers(ol)

### Control AXI GPIO IP using PYNQ libraries

In [ ]:
io_ctrl = ol.axi_gpio_0.channel1
io_load = ol.axi_gpio_0.channel2

count_mask = 0b01 # 1 bit
clear_mask = 0b10 # 1 bit

load_mask       = 0b11111111111111111 # 17 bits
load_value_mask = 0b01111111111111111 # 16 bits
load_en_mask    = 0b10000000000000000 #  1 bit

### Start / Stop Counter

In [ ]:
# Set Counter Enable HIGH
io_ctrl.write(0xFF, count_mask)

In [ ]:
# Set Counter Enable LOW
io_ctrl.write(0x00, count_mask)

### Clear / reset Counter

In [ ]:
# Stop Counter
io_ctrl.write(0x00, count_mask)

# Set Counter Clear HIGH
io_ctrl.write(0xFF, clear_mask)

time.sleep(0.01)

# Set Counter Clear LOW
io_ctrl.write(0x00, clear_mask)

### Load a value into counter

#### You'll need to change the settings of the ILAs as shown:

##### ILA 1:

![ILA1](ila1_settings.png) 

Set ILA 1 trigger: 'xs_16_load_en' ('R')( transition from 0->1 ) :

![ILA1_TRIG](ila1_trigger.png) 

##### ILA 2:

![ILA2](ila2_settings.png) 

Now arm Both ILA 2 followed by ILA 1 and run the following cell.

In [ ]:
# Start Counter
io_ctrl.write(0xFF, count_mask)

# Set Counter Load Value (16 bit value)
io_load.write( (0xDEAD | 0x10000 ), load_mask )

time.sleep(0.01)

# Set Counter Load EN LOW
io_load.write( (0xDEAD | 0x00000 ), load_mask)

# Stop Counter
io_ctrl.write(0x00, count_mask)

After running the above cell you should see that both of the ILA cores have triggered:

##### Hardwar Manager Waveform Output:

![ILA_WAVEFORMS_LOAD](ila_waveforms_load.png) 

In [ ]:
pxvc.stop_servers()